In [2]:
import soundfile as sf
import librosa
import ast
import numpy as np
import pandas as pd

In [4]:


with open('my_data/float_data_from_device.txt', 'r') as f:
   a = np.array([ast.literal_eval(line) for line in f][0], dtype='float32')

sf.write('my_data/float_data_from_device.wav', a, 44100)

pd.DataFrame(a).describe()


,0
count,220500.000000
mean,0.000008
std,0.003087
min,-0.035370
25%,-0.001312
50%,0.000031
75%,0.001343
max,0.029633


In [6]:
DEFAULT_FRAME_SIZE = 512
zcr_feat = librosa.feature.zero_crossing_rate(y=a, hop_length=DEFAULT_FRAME_SIZE)

print(len(zcr_feat[0]), len(a))

431 220500


In [22]:
sf.available_formats()

{'AIFF': 'AIFF (Apple/SGI)',
 'AU': 'AU (Sun/NeXT)',
 'AVR': 'AVR (Audio Visual Research)',
 'CAF': 'CAF (Apple Core Audio File)',
 'FLAC': 'FLAC (Free Lossless Audio Codec)',
 'HTK': 'HTK (HMM Tool Kit)',
 'SVX': 'IFF (Amiga IFF/SVX8/SV16)',
 'MAT4': 'MAT4 (GNU Octave 2.0 / Matlab 4.2)',
 'MAT5': 'MAT5 (GNU Octave 2.1 / Matlab 5.0)',
 'MPC2K': 'MPC (Akai MPC 2k)',
 'OGG': 'OGG (OGG Container format)',
 'PAF': 'PAF (Ensoniq PARIS)',
 'PVF': 'PVF (Portable Voice Format)',
 'RAW': 'RAW (header-less)',
 'RF64': 'RF64 (RIFF 64)',
 'SD2': 'SD2 (Sound Designer II)',
 'SDS': 'SDS (Midi Sample Dump Standard)',
 'IRCAM': 'SF (Berkeley/IRCAM/CARL)',
 'VOC': 'VOC (Creative Labs)',
 'W64': 'W64 (SoundFoundry WAVE 64)',
 'WAV': 'WAV (Microsoft)',
 'NIST': 'WAV (NIST Sphere)',
 'WAVEX': 'WAVEX (Microsoft)',
 'WVE': 'WVE (Psion Series 3)',
 'XI': 'XI (FastTracker 2)'}

In [23]:
sf.available_subtypes()

{'PCM_S8': 'Signed 8 bit PCM',
 'PCM_16': 'Signed 16 bit PCM',
 'PCM_24': 'Signed 24 bit PCM',
 'PCM_32': 'Signed 32 bit PCM',
 'PCM_U8': 'Unsigned 8 bit PCM',
 'FLOAT': '32 bit float',
 'DOUBLE': '64 bit float',
 'ULAW': 'U-Law',
 'ALAW': 'A-Law',
 'IMA_ADPCM': 'IMA ADPCM',
 'MS_ADPCM': 'Microsoft ADPCM',
 'GSM610': 'GSM 6.10',
 'G721_32': '32kbs G721 ADPCM',
 'G723_24': '24kbs G723 ADPCM',
 'DWVW_12': '12 bit DWVW',
 'DWVW_16': '16 bit DWVW',
 'DWVW_24': '24 bit DWVW',
 'VOX_ADPCM': 'VOX ADPCM',
 'DPCM_16': '16 bit DPCM',
 'DPCM_8': '8 bit DPCM',
 'VORBIS': 'Vorbis',
 'ALAC_16': '16 bit ALAC',
 'ALAC_20': '20 bit ALAC',
 'ALAC_24': '24 bit ALAC',
 'ALAC_32': '32 bit ALAC'}

In [78]:
#data, sr = librosa.load('my_data/flutter_sound_example.pcm', sr=44100, )

with open('my_data/flutter_sound_example.pcm', 'rb') as f:
    data, samplerate = sf.read(f, channels=1, samplerate=44100, subtype='PCM_16', format='RAW')


# Teste de audibilidade
sf.write('my_data/flutter_sound_example.wav', data, 44100)

# Funcionou! Agora vamos ver
pd.DataFrame(data).describe()

,0
count,109536.000000
mean,0.000119
std,0.036661
min,-0.389343
25%,-0.009216
50%,0.000000
75%,0.011292
max,0.247375
